In [7]:
# Pitch Prediction Data Cleaning Pipeline
# This notebook cleans and preprocesses pitch tracking data for machine learning
# Focus: Predicting pitch type (Fastball vs Other) based on game situation and sequence
#
# Import necessary libraries for data processing and database connectivity
import numpy as np                           # Numerical operations and array handling
import pandas as pd                          # Data manipulation and analysis
from sklearn.preprocessing import StandardScaler, LabelEncoder  # Data preprocessing tools
import mysql.connector                       # MySQL database connectivity
import os                                   # Operating system interface for environment variables
from dotenv import load_dotenv              # Load environment variables from .env file

# Load environment variables containing database credentials
# This keeps sensitive information out of the code
load_dotenv()

True

In [8]:
# Retrieve database connection parameters from environment variables
# This secure approach prevents hardcoding credentials in the notebook
DB_HOST = os.getenv('DB_HOST')           # Database server hostname/IP
DB_USER = os.getenv('DB_USER')           # Database username
DB_PASSWORD = os.getenv('DB_PASSWORD')   # Database password
DB_NAME = os.getenv('DB_NAME')           # Database name containing pitch tracking data

In [9]:
# Specify the target pitcher for analysis
# This model will be trained specifically for this pitcher's tendencies and patterns
pitcher = 'McCoy, Kyle'

In [10]:
# Database connectivity and data extraction functions
# These functions handle secure connections and pitcher-specific data retrieval

def create_connection():
    """
    Establish connection to MySQL database using environment variables
    
    Returns:
        connection object if successful, None if failed
    """
    try:
        connection = mysql.connector.connect(
            host=DB_HOST,
            user=DB_USER,
            password=DB_PASSWORD,
            database=DB_NAME
        )
        
        if connection.is_connected():
            print("Database connection established successfully")
            return connection
        
    except Exception as e:
        print(f'Database connection error: {e}')
        return None
    
def make_df_25(pitcher):
    """
    Extract 2025 season data for specified pitcher
    
    Features selected:
    - Game situation: Balls, Strikes, BatterSide
    - Pitch info: TaggedPitchType, PitchCall, RelSpeed
    - Pitcher info: Pitcher name, PitcherThrows
    
    Args:
        pitcher (str): Pitcher name to filter data
        
    Returns:
        DataFrame with 2025 season data or None if error
    """
    connection = create_connection()
    if connection is not None: 
        try:
            query = """
                    SELECT Pitcher, TaggedPitchType, PitchCall, Balls,
                    Strikes, RelSpeed, PitcherThrows, BatterSide
                    FROM 2025NCAATrackman
                    WHERE Pitcher = %s
                    """
            params = (pitcher,)
            df = pd.read_sql(query, connection, params=params)
            connection.close()
            print(f"Retrieved {len(df)} pitches from 2025 season")
            return df
        except Exception as e:
            print(f'Error retrieving 2025 data: {e}')
            if connection:
                connection.close()
    return None
    
def make_df_24(pitcher):
    """
    Extract 2024 season data for specified pitcher
    
    Note: Includes 'Batter' field for additional context in 2024 data
    
    Args:
        pitcher (str): Pitcher name to filter data
        
    Returns:
        DataFrame with 2024 season data or None if error
    """
    connection = create_connection()
    if connection is not None: 
        try:
            query = """
                    SELECT Pitcher, TaggedPitchType, PitchCall, Balls,
                    Strikes, RelSpeed, PitcherThrows, BatterSide, Batter
                    FROM 2024NCAATrackman
                    WHERE Pitcher = %s
                    """
            params = (pitcher,)
            df = pd.read_sql(query, connection, params=params)
            connection.close()
            print(f"Retrieved {len(df)} pitches from 2024 season")
            return df
        except Exception as e:
            print(f'Error retrieving 2024 data: {e}')
            if connection:
                connection.close()
    return None

In [11]:
# Extract and combine data from both seasons
# This creates a comprehensive dataset spanning multiple seasons for better model training

print("Extracting pitcher data from database...")
df_24 = make_df_24(pitcher)  # 2024 season data
df_25 = make_df_25(pitcher)  # 2025 season data

# Combine both seasons into a single dataset
# This provides more training examples and captures pitcher evolution over time
data = pd.concat([df_24, df_25], ignore_index=True)

print(f"Combined dataset created with {len(data)} total pitches")
print(f"Data spans from 2024 to 2025 seasons")

/var/folders/q5/0z0yqctd3xb1wgpb7qcl0g140000gn/T/ipykernel_77518/734578414.py:46: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query,connection,params=params)
/var/folders/q5/0z0yqctd3xb1wgpb7qcl0g140000gn/T/ipykernel_77518/734578414.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query,connection,params=params)
/var/folders/q5/0z0yqctd3xb1wgpb7qcl0g140000gn/T/ipykernel_77518/3112380430.py:3: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the re

In [101]:
# Create working copy of the data for cleaning operations
# This preserves the original raw data while allowing safe modifications
df = data.copy()

print("Sample of raw data:")
df.head()

,Pitcher,TaggedPitchType,PitchCall,Balls,Strikes,RelSpeed,PitcherThrows,BatterSide,Batter
0,"McCoy, Kyle",Fastball,FoulBallNotFieldable,0.0,0.0,91.88258,Left,Right,NaN
1,"McCoy, Kyle",Fastball,FoulBallNotFieldable,0.0,1.0,92.12607,Left,Right,NaN
2,"McCoy, Kyle",ChangeUp,BallCalled,0.0,2.0,86.01624,Left,Right,NaN
3,"McCoy, Kyle",ChangeUp,InPlay,1.0,2.0,85.64868,Left,Right,NaN
4,"McCoy, Kyle",Fastball,InPlay,0.0,0.0,91.03643,Left,Left,NaN


In [105]:
# Examine dataset dimensions
# Understanding the scale of data helps inform cleaning and modeling decisions
print(f"Dataset shape: {df.shape[0]} rows (pitches) × {df.shape[1]} columns (features)")
df.shape

(541, 9)

In [106]:
# Standardize pitch type classifications
# Different operators may use varying terminology - this creates consistency

print("Standardizing pitch type classifications...")
print("Before cleaning:", df['TaggedPitchType'].unique())

# Consolidate all fastball variants into single 'Fastball' category
# This simplifies the binary classification problem (Fastball vs Non-Fastball)
df['TaggedPitchType'] = df['TaggedPitchType'].str.replace('FourSeamFastBall', 'Fastball')
df['TaggedPitchType'] = df['TaggedPitchType'].str.replace('OneSeamFastball', 'Fastball')
df['TaggedPitchType'] = df['TaggedPitchType'].str.replace('TwoSeamFastBall', 'Fastball')
df['TaggedPitchType'] = df['TaggedPitchType'].str.replace('Four-Seam', 'Fastball')
df['TaggedPitchType'] = df['TaggedPitchType'].str.replace('Sinker', 'Fastball')
df['TaggedPitchType'] = df['TaggedPitchType'].str.replace('OneSeamFastBall', 'Fastball')

# Clean up problematic entries
df['TaggedPitchType'] = df['TaggedPitchType'].str.replace('Other', 'Undefined')
df['TaggedPitchType'] = df['TaggedPitchType'].str.replace(',', 'Undefined')  # Remove comma artifacts

print("After cleaning:", df['TaggedPitchType'].unique())
df['TaggedPitchType'].unique()

array(['Fastball', 'ChangeUp', 'Slider'], dtype=object)

In [107]:
# Examine pitch call outcomes before cleaning
# Understanding the variety of outcomes helps identify inconsistencies
print("Pitch call outcomes before cleaning:")
df['PitchCall'].unique()

array(['FoulBall', 'BallCalled', 'InPlay', 'StrikeSwinging',
       'StrikeCalled', 'HitByPitch'], dtype=object)

In [108]:
# Standardize pitch call outcomes
# Manual data entry leads to inconsistent terminology that needs cleaning

print("Cleaning pitch call terminology...")

# Consolidate ball outcomes
df['PitchCall'] = df['PitchCall'].str.replace('BallinDirt', 'BallCalled')
df['PitchCall'] = df['PitchCall'].str.replace('BallIntentional', 'BallCalled')
df['PitchCall'] = df['PitchCall'].str.replace('WildPitch', 'BallCalled')
df['PitchCall'] = df['PitchCall'].str.replace('ballCalled', 'BallCalled')  # Fix capitalization

# Remove incorrectly entered pitch types in PitchCall field
df['PitchCall'] = df['PitchCall'].str.replace('Slider', 'Undefined')
df['PitchCall'] = df['PitchCall'].str.replace('Sinker', 'Undefined')
df['PitchCall'] = df['PitchCall'].str.replace('Fastball', 'Undefined')

# Fix swing and miss variations
df['PitchCall'] = df['PitchCall'].str.replace('SwinginStrike', 'StrikeSwinging')
df['PitchCall'] = df['PitchCall'].str.replace('StrkeSwinging', 'StrikeSwinging')
df['PitchCall'] = df['PitchCall'].str.replace('StrikeSwinging ', 'StrikeSwinging')  # Remove trailing space

# Fix called strike variations
df['PitchCall'] = df['PitchCall'].str.replace('StirkeCalled', 'StrikeCalled')
df['PitchCall'] = df['PitchCall'].str.replace('StriekC', 'StrikeCalled')

# Consolidate foul ball outcomes
df['PitchCall'] = df['PitchCall'].str.replace('FoulBallNotFieldable', 'FoulBall')
df['PitchCall'] = df['PitchCall'].str.replace('FoulBallFieldable', 'FoulBall')

# Fix other outcome variations
df['PitchCall'] = df['PitchCall'].str.replace('Inplay', 'InPlay')
df['PitchCall'] = df['PitchCall'].str.replace('Hitbypitch', 'HitByPitch')
df['PitchCall'] = df['PitchCall'].str.replace('HitbyPitch', 'HitByPitch')
df['PitchCall'] = df['PitchCall'].str.replace('CatchersInterfernece', 'CatchersInterference')

print("Cleaned pitch call outcomes:")
df['PitchCall'].unique()

array(['FoulBall', 'BallCalled', 'InPlay', 'StrikeSwinging',
       'StrikeCalled', 'HitByPitch'], dtype=object)

In [109]:
# Create pitch groupings for sequence analysis
# Groups pitches by similar characteristics and strategic usage

print("Creating pitch group classifications...")

# Initialize with default value
df['PitchGroup'] = '0'

# Categorize pitches into strategic groups:
# FB = Fastball (primary velocity pitch)
# BB = Breaking Ball (sliders, curves, cutters - spin-heavy pitches)  
# OS = Offspeed (changeups, splitters - deception-based pitches)
df['PitchGroup'] = np.where(df['TaggedPitchType'] == 'ChangeUp', 'OS',
                    np.where(df['TaggedPitchType'] == 'Fastball', 'FB',
                    np.where(df['TaggedPitchType'] == 'Slider', 'BB',
                    np.where(df['TaggedPitchType'] == 'Curveball', 'BB',
                    np.where(df['TaggedPitchType'] == 'Cutter', 'BB',
                    np.where(df['TaggedPitchType'] == 'Splitter', 'OS', 'Other'))))))

print("Pitch group distribution:")
print(df['PitchGroup'].value_counts())

In [110]:
# Create indicator for 0-0 count (first pitch of at-bat)
# This helps identify and potentially exclude first pitches where sequence doesn't apply
df['0-0'] = np.where((df['Balls'] == 0) & (df['Strikes'] == 0), True, False)

print(f"First pitches (0-0 count): {df['0-0'].sum()} out of {len(df)} total pitches")

In [111]:
# Create sequence features: previous pitch characteristics
# These features capture pitch sequencing patterns that influence prediction

# Track the pitch group of the previous pitch
df['PreviousGroup'] = df['PitchGroup'].shift(1)

# Set previous velocity to '0' for first pitches (no previous pitch exists)
# Note: There appears to be a typo in the original - 'PreviousVelo' column doesn't exist
df.loc[(df['Balls'] == 0) & (df['Strikes'] == 0), 'PreviousGroup'] = '0'

print("Previous pitch group distribution:")
print(df['PreviousGroup'].value_counts())

In [112]:
# Track the outcome of the previous pitch
# Previous pitch results influence pitcher strategy for the next pitch

df['PreviousResult'] = df['PitchCall'].shift(1)

# Set previous result to '0' for first pitches
# Note: There's an inconsistency - creating 'PreviousCall' instead of updating 'PreviousResult'
df.loc[(df['Balls'] == 0) & (df['Strikes'] == 0), 'PreviousResult'] = '0'

print("Previous pitch result distribution:")
print(df['PreviousResult'].value_counts())

In [113]:
df.head()

,Pitcher,TaggedPitchType,PitchCall,Balls,Strikes,RelSpeed,PitcherThrows,BatterSide,Batter,PitchGroup,0-0,PreviousGroup,PreviousVelo,PreviousResult,PreviousCall
0,"McCoy, Kyle",Fastball,FoulBall,0.0,0.0,91.88258,Left,Right,NaN,FB,True,None,0,None,0
1,"McCoy, Kyle",Fastball,FoulBall,0.0,1.0,92.12607,Left,Right,NaN,FB,False,FB,NaN,FoulBall,NaN
2,"McCoy, Kyle",ChangeUp,BallCalled,0.0,2.0,86.01624,Left,Right,NaN,OS,False,FB,NaN,FoulBall,NaN
3,"McCoy, Kyle",ChangeUp,InPlay,1.0,2.0,85.64868,Left,Right,NaN,OS,False,OS,NaN,BallCalled,NaN
4,"McCoy, Kyle",Fastball,InPlay,0.0,0.0,91.03643,Left,Left,NaN,FB,True,OS,0,InPlay,0


In [114]:
# Create binary target variable and prepare final dataset
# Focus: Predict whether the next pitch will be a fastball or not

print("Creating target variable and final feature set...")

# Create binary target: 1 = Fastball, 0 = Other pitch types
df['Fastball'] = np.where(df['TaggedPitchType'] == 'Fastball', 1, 0)

print(f"Target distribution - Fastballs: {df['Fastball'].sum()}, Non-fastballs: {(df['Fastball'] == 0).sum()}")

# Remove first pitches (0-0 count) since sequence features don't apply
# First pitches lack previous pitch context that's crucial for prediction
df = df[df['0-0'] == False]
print(f"After removing 0-0 pitches: {len(df)} pitches remaining")

# Select final features for modeling
# These features capture: game situation + sequence context → pitch type prediction
clean_data = df[['Balls', 'Strikes', 'BatterSide', 'PreviousGroup', 'PreviousResult', 'Fastball']]

print(f"Final dataset shape: {clean_data.shape}")
print("Features selected: Game situation (Balls, Strikes, BatterSide) + Sequence (PreviousGroup, PreviousResult)")

In [115]:
clean_data

,Balls,Strikes,BatterSide,PreviousGroup,PreviousResult,Fastball
1,0.0,1.0,Right,FB,FoulBall,1
2,0.0,2.0,Right,FB,FoulBall,0
3,1.0,2.0,Right,OS,BallCalled,0
6,0.0,1.0,Right,FB,FoulBall,1
8,0.0,1.0,Left,BB,StrikeSwinging,0
...,...,...,...,...,...,...
535,2.0,0.0,Right,FB,BallCalled,1
536,2.0,1.0,Right,FB,StrikeCalled,0
537,3.0,1.0,Right,BB,BallCalled,1
538,3.0,2.0,Right,FB,StrikeCalled,1


In [116]:
# Convert categorical variables to dummy/indicator variables
# Neural networks require numerical input, so categorical features need encoding
print("Converting categorical variables to dummy variables...")

clean_data = pd.get_dummies(clean_data, columns=['Balls', 'Strikes', 'BatterSide', 'PreviousGroup', 'PreviousResult'])

print(f"After one-hot encoding: {clean_data.shape[1]} features")
print("Each categorical value becomes a separate binary feature")

In [94]:
# Convert boolean dummy variables to float32 for neural network compatibility
# PyTorch expects float32 inputs for optimal performance and memory usage
print("Converting boolean columns to float32...")

bool_columns = clean_data.select_dtypes(include='bool').columns
clean_data = clean_data.astype({col: 'float32' for col in bool_columns})

print(f"Converted {len(bool_columns)} boolean columns to float32")
print("Data types optimized for PyTorch neural network training")

In [95]:
# Export cleaned data for model training
# This preserves all preprocessing steps and creates a ready-to-use dataset
print("Exporting cleaned data...")

clean_data.to_csv('./clean_data.csv', index=False)

print("✅ Data cleaning complete!")
print(f"Saved clean_data.csv with {len(clean_data)} samples and {clean_data.shape[1]} features")
print(f"Ready for neural network training with fastball prediction target")